# Tools

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Tools-Practice")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Tools-Practice


In [3]:
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

In [4]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()

In [5]:
# 파이썬 코드를 실행하고 결과를 반환합니다.
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [7]:
# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=False):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)

In [8]:
# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [9]:
# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [10]:
# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

[1, 5, 19, 37, 40, 42]



In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    # 반환할 최대 검색 결과수
    max_results=6,
    
    # 원본 쿼리에 대한 짧은 답변 포함 여부
    include_answer=True,
    
    # 각 사이트의 정제된 HTML 콘텐츠 포함 여부
    include_raw_content=False,
    
    # 쿼리 관련 이미지 목록 포함 여부
    # include_images=True,
    
    # 검색 깊이
    # search_depth="advanced", # or "basic"
    
    # 검색 결과에 포함할 도메인 목록
    include_domains=["github.io", "wikidocs.net"],
    
    # 검색 결과에서 제외할 도메인 목록
    # exclude_domains = []
)

In [14]:
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'title': '01. 도구(Tools) - <랭체인LangChain 노트> - 위키독스',
  'url': 'https://wikidocs.net/262582',
  'content': '# os.environ["TAVILY_API_KEY"] = "TAVILY API 키 입력"\n\n```\n\n```\nfrom langchain_community.tools.tavily_search import TavilySearchResults\n\n# 도구 생성\ntool = TavilySearchResults(\n    max_results=6,\n    include_answer=True,\n    include_raw_content=True,\n    # include_images=True,\n    # search_depth="advanced", # or "basic"\n    include_domains=["github.io", "wikidocs.net"],\n    # exclude_domains = []\n)\n\n```\n\n```\n# 도구 실행\ntool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})\n\n``` [...] 1. <랭체인LangChain 노트> - Lang…\n2. CH16 에이전트(Agent)\n3. 01. 도구(Tools)\n\n1. 위키독스\n\n# 01. 도구(Tools)\n\n광고가 출력될 위치입니다.\n\n# 도구 (Tools)\n\n도구(Tool)는 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스입니다.\n\nLangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다.\n\nLangChain 에 통합된 도구 리스트는 아래 링크에서 확인할 수 있습니다.\n\n LangChain 통합된 도구 리스트\n\n```\n# API 키를 환경변수로 관리하기 위한 설정 

## 이미지 생성 도구

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=500)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke(
    {"image_desc": "매우 힘들게 코딩하는 개발자를 표현한 이미지"}
)

# 이미지 프롬프트 출력
print(image_prompt)

Create an image of a stressed-out software developer sitting at a cluttered desk filled with computer screens displaying lines of code, empty coffee cups, and scattered notes. The developer, a young Asian woman with disheveled hair, is furrowing her brow and biting her lip in concentration. She wears casual clothing, and her posture is hunched over as she types frantically on a mechanical keyboard. The room is dimly lit by the glow of the monitors, casting shadows on the walls adorned with motivational posters. In the background, a clock shows late hours, emphasizing the late-night coding struggle. Include a window with rain pouring outside, symbolizing the challenging atmosphere. The overall mood should reflect a chaotic but focused environment, capturing the intensity and dedication of coding under pressure.


In [2]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image
import os

# DALL-E API 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(
    model="dall-e-3",
    size="1024x1024",
    quality="standard",
    n=1,
)

# 질문
query = "매우 힘들게 코딩하는 개발자를 표현한 이미지"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지를 표시합니다.
Image(url=image_url, width=500)

In [3]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image
import os

# DALL-E API 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(
    model="dall-e-3",
    size="1024x1024",
    quality="standard",
    n=1,
)

# 질문
query = "매우 힘들게 코딩하다가 버그를 잡고 기뻐하는 개발자를 그린 이미지"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지를 표시합니다.
Image(url=image_url, width=500)

In [4]:
from langchain.tools import tool


# 데코레이터를 사용하여 함수를 도구로 변환합니다.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [6]:
# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})

7

In [5]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12

In [7]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

In [8]:
# 최신 뉴스 검색
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE9jZzB4T0Yzbkh2UGFvM3M3ZHVJZDFEYV9Tc1JEM0tHTlVqejE4MEhTd25tTE9WZDdNWDN5X3pLRjFTTlFwdDVUTUNxWHdsNVp3ZVhMRTJzOUN6UdIBVEFVX3lxTFBrTzZwMEgzM0VORTZ2T2RXd2k3WlNFYWhEYmsyWDJJN0piai1xNjJTWmF5S05acURJNlZLNWR2eGp0ZGlyUmNTc3Jhd2ZDSlVFd0R2Rw?oc=5',
  'content': '정부는 \'자진 출국\'이라는데…美 장관 "한국인 추방" 표현 [이상은의 워싱턴나우] - 한국경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBrZkQ0NklGXzlqNVo1NkpPOXMyX3FSdUZWWENGdGRPYnFSRnZEYWVkMFJXckVmYXpRVFgzX3pDM2ZSdXAxYlhGR3M3dkRvMk1UUXpzQjdPbFAzZ9IBX0FVX3lxTE1UUnpJWE1fVTdEUVRvMVJMODNYblB1RDgzSHduWEJKN01mb0JfUHg0QmtpUVlqLS1ianprUTZZaDBtNmdxVEdCWkZQUWc2OVc4WGJxeTZ2OENaWDNXQmtR?oc=5',
  'content': '특검 “권성동, 비서관 통해 공범에 몰래 접촉 시도···증거인멸 우려” - 경향신문'},
 {'url': 'https://news.google.com/rss/articles/CBMia0FVX3lxTE9xc2VtWldCSnFUZk5xN214WFdwX3d4c3pMVWhuX2ZwZUlsaXdsVHdyaGdZeHp2b3IzaWZwSGt6T1hJYWdDMUhULUxuNm9BVFRXcWVHVXBRejFnNEcxTGhXbnJjZkV3ZXdwSGRR?oc=5',
  'content': '이 대통령, 권성동 압력 의혹 필리핀 차관사업 “즉시 중지 명령” - 

In [9]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("개발자", k=3)

[{'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE91YjJEc0xQcUpSeW81RlVEUHliZmE0ZFRhVzJtOERtS1lEck9pM0VzakJuY1ViQkZaQ01PQk5IeW13MUp1eW5tSldDR3ZvSUJlVkFteG1R?oc=5',
  'content': "저무는 코딩 불패, 설 자리 잃은 초급 개발자, AI발 구조조정…판교 IT 대기업들 '해고 바람' - 중앙일보"},
 {'url': 'https://news.google.com/rss/articles/CBMic0FVX3lxTFB4d2NyMlJkRHh6bnhjaXYzbFFQQlI0RUNhd3hWYzJYemE0THM3TnhlZFF5WlkwZ1prdW5OOFBiczJhVDc3c2JxSjY3eGdWLU0tXzRVT2lZRjhDTkc2cXVuaHlMRWg2WWhEY0xxa0J4SXpuZHM?oc=5',
  'content': "안드로이드 대표 런처 '노바 런처' 역사 속으로…마지막 개발자 떠나 - 디지털투데이"},
 {'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE9tY3VMU2QtT21xd2Z4N2NPWVU5Q2VTMldmY0MwMzNBU2hGSjFOc2xuelc2azNNT1Q1MnFKM2FlWUxsUTE4Y2F6N0xMVGZmZjNVUU9XcnJPYTVFUE9wSmNiTEs2a01PbGdaSVFpNXdB?oc=5',
  'content': "[UDC 2025] 두나무, 웹3 인프라 브랜드 '기와' 공개…국내 개발자 글로벌 무대로 - 뉴스후플러스"}]

In [10]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [11]:
# 실행 결과
search_keyword.invoke({"query": "LangChain AI"})

LangChain AI


[{'url': 'https://news.google.com/rss/articles/CBMiigFBVV95cUxNckhZYnFjaDRYUHR2ZlpnTC0yR2FJWDNYSmFzUUVkbE1JSGNHRjZoSVZvemFKbGhBVDVfUlhrVXpmU3cyR1ZhSDVCYkpOYTRMQi1icEQ1aFl2OG5ZV2Z6M044SmxaWEY1Ylhmck1DRXNMWW9RVHZiQjNhVkV5QlUtM1NmcGhYdERVYXc?oc=5',
  'content': 'Db2 LangChain Connector 발표 - IBM'},
 {'url': 'https://news.google.com/rss/articles/CBMic0FVX3lxTE5QeGFoOTJqRU53c3duWExsaS1xWEdlVWxHaEhCU2RDUkZsZDNQT2NQZ1JKRkpWVHQ4aTVLUmNaM0YwSk9KcmFwckdpLWhEYjVXMDhsX2dlZU9IU0pISlJVLVhTSzVWRUtHU29MOGhQSUFLaDQ?oc=5',
  'content': '랭체인, AI 에이전트 설계 혁신…개발자 통합 지원 강화 - 디지털투데이'},
 {'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5',
  'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'},
 {'url': 'https://news.google.com/rss/articles/CBMicEFVX3lxTE5TTWFjck9DZEVfTGNDNDJSTUR0LTZfQUotMGNsNUpOdkxTZEU4b0dvVnBqU0VJbWlsUkNhZTNnbDVFT29KRVRVZ1ZLS3